In [1]:
import cv2
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope
import keras
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.
c:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synon

In [2]:
def relu6(x):
    return K.relu(x, max_value=6)

In [3]:
with CustomObjectScope({'relu6': relu6,'DepthwiseConv2D': keras.layers.DepthwiseConv2D}):
    model = load_model('my_model.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [4]:
def read_img1(frame, crop_specs, input_sz=(224, 224)):
    """
    Read image file and return cropped, resized, intensity-normalized array
        Arguments
            :img_path: str
            :crop_specs: list
            :input_sz: tuple
        Output
            :rgb_norm: numpy array
    """
#     bgr = frame
#     rgb = bgr[:,:,::-1]
    rgb = frame
    if len(crop_specs) == 4:
        rgb = rgb[ crop_specs[0]:crop_specs[1], crop_specs[2]:crop_specs[3], :]
    rgb = cv2.resize(rgb, (600, 800))
    rgb = cv2.resize(rgb, input_sz)
    rgb_norm = rgb.astype(np.float32)/ 255.
    
    return rgb_norm

In [5]:
cap = cv2.VideoCapture(0)
if cap.isOpened() == False:
    print('unable to read camera feed')
out = cv2.VideoWriter('webcam.avi', cv2.VideoWriter_fourcc(*'XVID'), 10, (640, 480))

In [6]:
while True:
    ret, frame = cap.read()
    if ret:
        rows, cols, _ = frame.shape
        crop_specs = [int(0.05*rows), int(0.95*rows), int(0.05*cols), int(0.95*cols)]
        image_rz = read_img1(frame, crop_specs)
        cv2.imshow('img', image_rz)
        img = np.array([image_rz])
        output = model.predict(img)
        mask0 = np.squeeze(output)
        mask = np.expand_dims(mask0, axis=2)
        combo = mask*image_rz
        cv2.imshow('output', combo)
        # writing video
#         out.write(mask)
        cv2.imshow('Mask', mask)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [7]:
frame.shape

(480, 640, 3)